In [2]:
import redis
from mlrepricer import redisdb, helper, parser, setup, updateprice
import xmltodict

In [3]:
helper.rediscred

{'host': '127.0.0.1', 'port': '6379'}

In [5]:
r = redis.StrictRedis(**helper.rediscred, decode_responses=True)
redisdb.server_start()

In [6]:
asin_gen = r.scan_iter(match=r'[a-zA-Z0-9]'*10)

In [14]:
asin = next(asin_gen)
print(asin)

B0776F2W4P


In [15]:
m = r.zrevrangebyscore(asin, 'inf', '-inf', start=0, num=1,
                           withscores=True)
x = parser.parse(eval(m[0][0]))

In [17]:
message_df = x

In [28]:
message_df

,asin,feedback,feedbackpercent,instock,isbuyboxwinner,isfeaturedmerchant,isprime,price,sellerid,shipping_maxhours,shipping_minhours,time_changed
0,B0776F2W4P,275,99,1,0,1,0,7.85,ADWP9I1S82QC2,48,24,2019-01-07 14:36:12.737
1,B0776F2W4P,122,95,1,0,1,0,7.95,A1VCF5T6XZWY3C,48,24,2019-01-07 14:36:12.737
2,B0776F2W4P,5101,99,1,1,1,1,8.15,A1SAM4LDA4SAIA,0,0,2019-01-07 14:36:12.737
3,B0776F2W4P,1705,98,1,0,1,0,8.39,A2EHWGAW6J9W8Q,48,24,2019-01-07 14:36:12.737
4,B0776F2W4P,9069,98,1,0,1,0,8.39,ADBIN7JOKZABU,48,24,2019-01-07 14:36:12.737
5,B0776F2W4P,2420,100,1,0,1,1,8.90,A3L0FT1E49IH4L,0,0,2019-01-07 14:36:12.737
6,B0776F2W4P,2420,100,1,0,1,1,8.90,A3L0FT1E49IH4L,0,0,2019-01-07 14:36:12.737
7,B0776F2W4P,1820,97,1,0,1,0,9.89,A3COZDX540BPM9,24,24,2019-01-07 14:36:12.737
8,B0776F2W4P,4803,99,1,0,1,1,9.90,A29GY95S72SVCC,0,0,2019-01-07 14:36:12.737
9,B0776F2W4P,9069,98,1,0,1,0,10.39,ADBIN7JOKZABU,48,24,2019-01-07 14:36:12.737


In [35]:
for x in message_df.iterrows():
    print(x[0])
    x = x[1]
    if (x['isbuyboxwinner']):
        print('happy')
    elif x['isbuyboxwinner'] & x['isprime']:
        prime_winner.append(x)
    elif x['isbuyboxwinner'] & ~x['isprime']:
        nonprime_winner.append(x)

0
1
2
happy
3
4
5
6
7
8
9
10
11


In [34]:
nonprime_winner

NameError: name 'nonprime_winner' is not defined

In [45]:
# example get value by key for 
# type: specified members with the specified scores to the sorted set stored at key
m = r.zscan(next(m))

In [67]:
# m1 is where all messages are, the next 0 is where the latest message is stored (highest timestamp value)
# the next 0 is getting the stored data a string which is the message how we get it from the aws queue
x = m[1][0][0]
xmltodict.parse(eval(x)[0]['Body'])
#eval(x)[0]['ReceiptHandle']

OrderedDict([('Notification',
              OrderedDict([('NotificationMetaData',
                            OrderedDict([('NotificationType',
                                          'AnyOfferChanged'),
                                         ('PayloadVersion', '1.0'),
                                         ('UniqueId',
                                          '1ff117ca-a994-4f9d-a5a0-fba66adabc05'),
                                         ('PublishTime',
                                          '2019-01-05T04:35:31.005Z'),
                                         ('SellerId', 'A2EHWGAW6J9W8Q'),
                                         ('MarketplaceId',
                                          'A1PA6795UKMFR9')])),
                           ('NotificationPayload',
                            OrderedDict([('AnyOfferChangedNotification',
                                          OrderedDict([('OfferChangeTrigger',
                                                        Ordered

Example iterate over all keys

In [11]:
def asin_generator():
    """
    Match keys which contain all messages per asin.
    Key example: B07B9NGM2J
    Return a generator of keys you can use to get the actual messages.
    """
    return r.scan_iter(match=r'[a-zA-Z0-9]'*10)

In [14]:
def actionable_asins():
    """
    Get asin we have to take actions.
    one key: updated_asins
    """
    return r.sscan_iter('updated_asins')

In [15]:
def parsed_messages(asin):
    """Make a list with all messages parsed."""
    mlist = r.zrevrangebyscore(asin, 'inf', '-inf')
    return [parser.parse(eval(message)) for message in mlist]

In [16]:
#[x for x in message_key()]

In [17]:
result = []
asin_gen = asin_generator()
for asin in asin_gen:
    result.append(parsed_messages(asin))

In [18]:
result[33][1]

IndexError: list index out of range

In [19]:
updateprice.get_latest_message(asin)

NameError: name 'asin' is not defined